In [1]:
import os
from binascii import hexlify
from time import sleep
import datetime

In [2]:
import io

In [3]:
tagdir       = '/home/pi/tags/1fbcd99fd29f/tag'
testdir      = tagdir+'/.test'
zerofile     = testdir+'/zero'
onesfile     = testdir+'/ones'
echofile     = testdir+'/echo'

In [4]:
import tagnet

TagNet Driver Version 0.2.1


In [5]:
!pwd

/mnt/neptune/TagNet/notebooks/TagFuse_Tutorial


In [6]:
def int32(x):
  if x>0xFFFFFFFF:
    raise OverflowError
  if x>0x7FFFFFFF:
    x=int(0x100000000-x)
    if x<2147483648:
      return -x
    else:
      return -2147483648
  return x

In [7]:
def round_to_word(a):
    return ((a-(a%4)) if (a%4) else (a))

In [8]:
def get_file_size(fn):
    ret=!stat  -c '%s' $fn
    offset=int(ret[0])
    int32(offset)
    return offset

In [9]:
def err_by_value(val):
    for e in tagnet.tlv_errors:
        if e.value == val: break
    return e.name

In [10]:
def size_to_err(pos):
    ipos = int32(pos)
    if ipos < 0:
        print('error', -ipos, err_by_value(-ipos))
        return -ipos
    return ipos

#  Show Test Directory contents

In [11]:
!ls -al $testdir

total 0
drwxr-x--x 10 pi pi   0 Dec 31  1969 .
drwxr-x--x  8 pi pi   0 Dec 31  1969 ..
-rw-rw--w-  1 pi pi   0 Nov 16 14:35 drop
-rw-rw--w-  1 pi pi   0 Nov 16 14:35 echo
-rw-rw-r--  1 pi pi  40 Nov 16 14:35 ones
-rw-rw--w-  1 pi pi 179 Nov 16 14:35 rssi
-rw-rw--w-  1 pi pi 155 Nov 16 14:35 rx_rssi
-rw-rw-r--  1 pi pi   0 Dec 31  1969 sum
-rw-rw--w-  1 pi pi   0 Dec 31  1969 tx_pwr
-rw-rw-r--  1 pi pi  10 Nov 16 14:35 zero


# Run Tests on Zero and Ones Byte Streams

Use hexdump to verify the data (it automatically compares to remove duplicate lines).

NOTE: hexdump uses buffered file system, so tagfuse read will be for 512

In [12]:
!truncate --size 0 $zerofile

In [13]:
!ls -al $zerofile

-rw-rw-r-- 1 pi pi 0 Nov 16 14:35 /home/pi/tags/1fbcd99fd29f/tag/.test/zero


In [14]:
!hexdump -n 5120 $zerofile

0000000 0000 0000 0000 0000 0000 0000 0000 0000
*
0001400


In [15]:
!ls -al $zerofile

-rw-rw-r-- 1 pi pi 6080 Nov 16 14:36 /home/pi/tags/1fbcd99fd29f/tag/.test/zero


In [16]:
!truncate --size 0 $onesfile

In [17]:
!ls -al $onesfile

-rw-rw-r-- 1 pi pi 0 Nov 16 14:36 /home/pi/tags/1fbcd99fd29f/tag/.test/ones


In [18]:
!hexdump -n 5120 $onesfile

0000000 ffff ffff ffff ffff ffff ffff ffff ffff
*
0001400


In [19]:
!ls -al $onesfile

-rw-rw-r-- 1 pi pi 6400 Nov 16 14:37 /home/pi/tags/1fbcd99fd29f/tag/.test/ones


#  Test Write Zero and Ones to Tags

## Start with zeros files

In [20]:
!truncate --size 0 $zerofile

In [21]:
!ls -al $zerofile

-rw-rw-r-- 1 pi pi 0 Nov 16 14:37 /home/pi/tags/1fbcd99fd29f/tag/.test/zero


In [22]:
fn = io.FileIO(zerofile, 'w+b')

In [23]:
fn.write('\x00' * 10)

10L

In [24]:
!ls -al $zerofile

-rw-rw-r-- 1 pi pi 20 Nov 16 14:37 /home/pi/tags/1fbcd99fd29f/tag/.test/zero


In [25]:
fn.close()

In [26]:
!ls -al $zerofile

-rw-rw-r-- 1 pi pi 20 Nov 16 14:37 /home/pi/tags/1fbcd99fd29f/tag/.test/zero


## repeat for ones files

In [27]:
!truncate --size 0 $onesfile

In [28]:
!ls -al $onesfile

-rw-rw-r-- 1 pi pi 0 Nov 16 14:37 /home/pi/tags/1fbcd99fd29f/tag/.test/ones


In [29]:
fn = io.FileIO(onesfile, 'w+b')

In [30]:
fn.write('\x00' * 10)

0L

In [31]:
!ls -al $onesfile

-rw-rw-r-- 1 pi pi 40 Nov 16 14:37 /home/pi/tags/1fbcd99fd29f/tag/.test/ones


In [32]:
fn.close()

In [33]:
!ls -al $onesfile

-rw-rw-r-- 1 pi pi 40 Nov 16 14:37 /home/pi/tags/1fbcd99fd29f/tag/.test/ones


# Run Echo Test

First, write the echo pattern to the tag, then read it back n times.

In [34]:
!echo 'new string of data' > $echofile

In [35]:
!ls -al $echofile

-rw-rw--w- 1 pi pi 0 Nov 16 14:37 /home/pi/tags/1fbcd99fd29f/tag/.test/echo


In [36]:
!hexdump -C -n 512 $echofile

00000000  6e 65 77 20 73 74 72 69  6e 67 20 6f 66 20 64 61  |new string of da|
00000010  74 61 0a 6e 65 77 20 73  74 72 69 6e 67 20 6f 66  |ta.new string of|
00000020  20 64 61 74 61 0a 6e 65  77 20 73 74 72 69 6e 67  | data.new string|
00000030  20 6f 66 20 64 61 74 61  0a 6e 65 77 20 73 74 72  | of data.new str|
00000040  69 6e 67 20 6f 66 20 64  61 74 61 0a 6e 65 77 20  |ing of data.new |
00000050  73 74 72 69 6e 67 20 6f  66 20 64 61 74 61 0a 6e  |string of data.n|
00000060  65 77 20 73 74 72 69 6e  67 20 6f 66 20 64 61 74  |ew string of dat|
00000070  61 0a 6e 65 77 20 73 74  72 69 6e 67 20 6f 66 20  |a.new string of |
00000080  64 61 74 61 0a 6e 65 77  20 73 74 72 69 6e 67 20  |data.new string |
00000090  6f 66 20 64 61 74 61 0a  6e 65 77 20 73 74 72 69  |of data.new stri|
000000a0  6e 67 20 6f 66 20 64 61  74 61 0a 6e 65 77 20 73  |ng of data.new s|
000000b0  74 72 69 6e 67 20 6f 66  20 64 61 74 61 0a 6e 65  |tring of data.ne|
000000c0  77 20 73 74 72 69 6e 67  20 6f 66 20 64 61

In [37]:
!ls -al $echofile

-rw-rw--w- 1 pi pi 626 Nov 16 14:37 /home/pi/tags/1fbcd99fd29f/tag/.test/echo


In [38]:
!truncate --size 0 $echofile

In [39]:
!ls -al $echofile

-rw-rw--w- 1 pi pi 0 Nov 16 14:37 /home/pi/tags/1fbcd99fd29f/tag/.test/echo


In [53]:
!xxd -l 512 -c 19 $echofile

00000000: 6e65 7720 7374 7269 6e67 206f 6620 6461 7461 0a  new string of data.
00000013: 6e65 7720 7374 7269 6e67 206f 6620 6461 7461 0a  new string of data.
00000026: 6e65 7720 7374 7269 6e67 206f 6620 6461 7461 0a  new string of data.
00000039: 6e65 7720 7374 7269 6e67 206f 6620 6461 7461 0a  new string of data.
0000004c: 6e65 7720 7374 7269 6e67 206f 6620 6461 7461 0a  new string of data.
0000005f: 6e65 7720 7374 7269 6e67 206f 6620 6461 7461 0a  new string of data.
00000072: 6e65 7720 7374 7269 6e67 206f 6620 6461 7461 0a  new string of data.
00000085: 6e65 7720 7374 7269 6e67 206f 6620 6461 7461 0a  new string of data.
00000098: 6e65 7720 7374 7269 6e67 206f 6620 6461 7461 0a  new string of data.
000000ab: 6e65 7720 7374 7269 6e67 206f 6620 6461 7461 0a  new string of data.
000000be: 6e65 7720 7374 7269 6e67 206f 6620 6461 7461 0a  new string of data.
000000d1: 6e65 7720 7374 7269 6e67 206f 6620 6461 7461 0a  new string of data.
000000e4: 6e65 7720 7374 7269 6e67 206f 6620 6461 74

In [54]:
!ls -al $echofile

-rw-rw--w- 1 pi pi 1706 Nov 16 14:39 /home/pi/tags/1fbcd99fd29f/tag/.test/echo


In [42]:
fn = io.FileIO(echofile, 'w+b')

In [43]:
fn.read(10)

'new string'

In [44]:
fn.tell()

10L

In [45]:
!ls -al $echofile

-rw-rw--w- 1 pi pi 10 Nov 16 14:37 /home/pi/tags/1fbcd99fd29f/tag/.test/echo


In [46]:
fn.truncate(0)

0

In [47]:
!ls -al $echofile

-rw-rw--w- 1 pi pi 0 Nov 16 14:37 /home/pi/tags/1fbcd99fd29f/tag/.test/echo


In [48]:
fn.close()